In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from pyod.models.iforest import IForest
# from pyod.models.gmm import GMM
# from sklearn.svm import OneClassSVM
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense
# import numpy as np
# from sklearn.model_selection import train_test_split


In [2]:
# import requests

# site_id = "site_104"  
# params = "pm10cnc,pm2.5cnc"
# startdate = "2021-11-01T00:00"
# enddate = "2025-03-13T00:00"

# # API URL
# api_url = f"http://atmos.urbansciences.in/adp/v4/getDeviceDataParam/imei/{site_id}/params/{params}/startdate/{startdate}/enddate/{enddate}/ts/mm/avg/30/api/63h3AckbgtY?gaps=1&gap_value=NaN"

# # Make request
# response = requests.get(api_url)

# # Check if request was successful
# if response.status_code == 200:
#     # Save CSV file
#     filename = f"{site_id}_data.csv"
#     with open(filename, "wb") as file:
#         file.write(response.content)
#     print(f"CSV file saved as: {filename}")
# else:
#     print(f"Error: API request failed with status code {response.status_code}")


In [3]:
import pandas as pd
import numpy as np

def clean_and_prepare(df):
    """
    Cleans and prepares the dataframe:
    - Ensures datetime index
    - Converts numeric data before interpolation
    - Creates time-based features (hour, sin_hour, cos_hour)
    - Adds lag features for pm2.5cnc and pm10cnc
    - Handles missing values correctly
    """

    if isinstance(df.index, pd.MultiIndex):
        dt_index = pd.to_datetime(df.index.get_level_values(0), errors="coerce")
        df.index = df.index.set_levels(dt_index, level=0)
    else:
        if "dt_time" in df.columns:
            df["dt_time"] = pd.to_datetime(df["dt_time"], errors="coerce")
            df.dropna(subset=["dt_time"], inplace=True)
            df.set_index("dt_time", inplace=True)

    if "pm2_5cnc" not in df.columns or "pm10cnc" not in df.columns:
        print(f"Warning: One or both pollutant columns are missing! Columns available: {df.columns}")

    for col in ["pm2_5cnc", "pm10cnc"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df = df[numeric_cols]

    if isinstance(df.index, pd.MultiIndex) and "site_id" in df.columns:
        df.set_index("site_id", append=True, inplace=True)

    if not df.empty:
        df.interpolate(method="time", inplace=True)

    df["hour"] = df.index.get_level_values(0).hour
    df["sin_hour"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["cos_hour"] = np.cos(2 * np.pi * df["hour"] / 24)

    for col in ["pm2.5cnc", "pm10cnc"]:
        if col in df.columns:
            df[f"{col}_lag1"] = df[col].shift(1).bfill()
            df[f"{col}_lag2"] = df[col].shift(2).bfill()

    df["month"] = df.index.get_level_values(0).month

    def get_season(month):
        if month in [1, 2]: return "Winter"
        elif month in [6, 7, 8, 9]: return "Monsoon"
        else: return "Post-Monsoon"

    df["season"] = df["month"].apply(get_season)
    df["season_code"] = pd.Categorical(df["season"]).codes
    df.drop(columns=["season"], inplace=True)

    df.fillna(method="ffill", inplace=True)
    df.fillna(method="bfill", inplace=True)

    if "pm2_5cnc" not in df.columns:
        raise ValueError("ERROR: 'pm2.5cnc' column is missing from DataFrame after processing!")

    return df


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def encode_pm25(pm25):
    """
    Encodes PM2.5 value into a numeric score:
      1: Good (0–30 µg/m³)
      2: Satisfactory (31–60 µg/m³)
      3: Moderate (61–90 µg/m³)
      4: Poor (91–120 µg/m³)
      5: Very Poor (121–250 µg/m³)
      6: Severe (251 µg/m³ and above)
    """
    if pm25 <= 30:
        return 1
    elif pm25 <= 60:
        return 2
    elif pm25 <= 90:
        return 3
    elif pm25 <= 120:
        return 4
    elif pm25 <= 250:
        return 5
    else:
        return 6

def encode_pm10(pm10):
    """
    Encodes PM10 value into a numeric score:
      1: Good (0–50 µg/m³)
      2: Satisfactory (51–100 µg/m³)
      3: Moderately Polluted (101–250 µg/m³)
      4: Poor (251–350 µg/m³)
      5: Very Poor (351–430 µg/m³)
      6: Severe (431 µg/m³ and above)
    """
    if pm10 <= 50:
        return 1
    elif pm10 <= 100:
        return 2
    elif pm10 <= 250:
        return 3
    elif pm10 <= 350:
        return 4
    elif pm10 <= 430:
        return 5
    else:
        return 6

def combined_air_quality_label(pm25, pm10):
    """
    Combines PM2.5 and PM10 numeric scores by taking the floor of the average.
    For example, if one pollutant is 'Severe' (6) and the other 'Moderate' (3),
    the average (4.5) is floored to 4, which we interpret as 'Poor'.
    """
    score_pm25 = encode_pm25(pm25)
    score_pm10 = encode_pm10(pm10)
    combined_score = (score_pm25 + score_pm10) // 2
    return combined_score


combined_labels = {
    1: "Good",
    2: "Satisfactory",
    3: "Moderate",
    4: "Poor",
    5: "Very Poor",
    6: "Severe"
}

def process_air_quality_data(df):
    """
    Processes a DataFrame containing 'pm25' and 'pm10' columns:
      - Encodes each pollutant into its numeric score.
      - Creates a combined score.
      - Retains only numeric columns.
    """
    df['pm25_numeric'] = df['pm25'].apply(encode_pm25)
    df['pm10_numeric'] = df['pm10'].apply(encode_pm10)
    df['combined_numeric'] = df.apply(
        lambda row: combined_air_quality_label(row['pm25'], row['pm10']),
        axis=1
    )
    df['combined_label'] = df['combined_numeric'].map(combined_labels)
    
    # Keep only numeric columns (if desired)
    df_numeric = df.select_dtypes(include=[np.number])
    return df_numeric



def check_misclassification(row, pollutant):
    """
    Identifies if the anomaly detection for a row is a misclassification.
    
    Logic:
      - Computes the combined numeric air quality label using both 'pm2.5cnc' and 'pm10cnc' columns.
      - If an anomaly is flagged (column value 1) but the combined label is in a milder category (1, 2, or 3),
        it is a misclassification.
      - If no anomaly is flagged (0) but the combined label is in a worse category (4, 5, or 6),
        it is a misclassification.
    
    Returns:
      - True if misclassification, False otherwise.
    """
    # Compute combined numeric label from the pollutant values (assumes both columns are available)
    combined = combined_air_quality_label(row["pm2.5cnc"], row["pm10cnc"])
    
    # Determine the anomaly flag column (e.g., "Anomaly_pm25cnc_features" for "pm2.5cnc")
    anomaly_col = f"Anomaly_{pollutant.replace('.', '')}"
    anomaly_flag = row[anomaly_col]
    
    if anomaly_flag == 1 and combined in [1, 2, 3]:
        return True
    # elif anomaly_flag == 0 and combined in [4, 5, 6]:
    #     return True
    else:
        return False

In [5]:
def run_isoforest(train_data):
    """
    Trains an Isolation Forest on the given training data.
    Converts detected anomalies (-1) to normal (1) only if they fall within safe AQI ranges:
      - For 'pm2.5cnc': safe range is 0 to 90.
      - For 'pm10cnc': safe range is 0 to 350.
    Returns: (model, scores, labels)
    """
    # Train the model
    model = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
    model.fit(train_data)
    
    # Get outputs from Isolation Forest
    scores = model.decision_function(train_data)  # Higher => more normal
    labels = model.predict(train_data)              # 1 => normal, -1 => anomaly

    # safe_range_mask = None
    # if 'pm2.5cnc' in train_data.columns and 'pm10cnc' in train_data.columns:
    #     mask_pm25 = (train_data["pm2.5cnc"] >= 0) & (train_data["pm2.5cnc"] <= 90)
    #     mask_pm10 = (train_data["pm10cnc"] >= 0) & (train_data["pm10cnc"] <= 350)
    #     safe_range_mask = mask_pm25 & mask_pm10
    # elif 'pm2.5cnc' in train_data.columns:
    #     safe_range_mask = (train_data["pm2.5cnc"] >= 0) & (train_data["pm2.5cnc"] <= 90)
    # elif 'pm10cnc' in train_data.columns:
    #     safe_range_mask = (train_data["pm10cnc"] >= 0) & (train_data["pm10cnc"] <= 350)
    
    # # For rows flagged as anomaly (-1) but within the safe range, force them to normal (1)
    # if safe_range_mask is not None:
    #     labels = np.where((labels == -1) & safe_range_mask, 1, labels)
    
    return model, scores, labels


# def run_svm(train_data):
#     """
#     Trains a One-Class SVM on the given training data.
#     Converts detected anomalies (-1) to normal (1) only if they fall within safe AQI ranges:
#       - For 'pm2.5cnc': safe range is 0 to 90.
#       - For 'pm10cnc': safe range is 0 to 350.
#     Returns: (model, scores, labels)
#     """
#     # Train the model
#     model = OneClassSVM(nu=0.05, kernel="rbf", gamma="scale")
#     model.fit(train_data)
    
#     # Get outputs from One-Class SVM
#     scores = model.decision_function(train_data)  # Higher => more normal
#     labels = model.predict(train_data)              # 1 => normal, -1 => anomaly

#     # Define safe range mask based on available pollutant columns
#     safe_range_mask = None
#     if 'pm2.5cnc' in train_data.columns and 'pm10cnc' in train_data.columns:
#         mask_pm25 = (train_data["pm2.5cnc"] >= 0) & (train_data["pm2.5cnc"] <= 250)
#         mask_pm10 = (train_data["pm10cnc"] >= 0) & (train_data["pm10cnc"] <= 350)
#         safe_range_mask = mask_pm25 & mask_pm10
#     elif 'pm2.5cnc' in train_data.columns:
#         safe_range_mask = (train_data["pm2.5cnc"] >= 0) & (train_data["pm2.5cnc"] <= 250)
#     elif 'pm10cnc' in train_data.columns:
#         safe_range_mask = (train_data["pm10cnc"] >= 0) & (train_data["pm10cnc"] <= 350)
    
#     # For rows flagged as anomaly (-1) but within the safe range, force them to normal (1)
#     if safe_range_mask is not None:
#         labels = np.where((labels == -1) & safe_range_mask, 1, labels)
    
#     return model, scores, labels

def split_data_random(df):
    """
    Splits the dataset into train (70%), test (20%), and validation (10%).

    Parameters:
    df (DataFrame): The dataset to split.

    Returns:
    train (DataFrame): 70% of data for training.
    test (DataFrame): 20% of data for testing.
    validation (DataFrame): 10% of data for final validation.
    """
    train, temp = train_test_split(df, test_size=0.3, random_state=42)  # 70% train, 30% temp
    test, validation = train_test_split(temp, test_size=1/3, random_state=42)  # 20% test, 10% validation
    return train, test, validation

def split_data(df):
    """
    Splits the dataset into train (70%), test (20%), and validation (10%) 
    in a historical order (earliest to latest).
    
    Parameters:
    df (DataFrame): The dataset to split. Assumes it is sorted by timestamp.
    
    Returns:
    train (DataFrame): First 70% of data for training.
    test (DataFrame): Next 20% of data for testing.
    validation (DataFrame): Final 10% of data for validation.
    """
    # Ensure data is sorted by timestamp (assuming a 'timestamp' column exists)
    df = df.sort_values(by='dt_time')

    # Compute split indices
    n = len(df)
    train_end = int(n * 0.7)
    test_end = int(n * 0.9)  # 70% train + 20% test = 90%, remaining 10% is validation

    # Split sequentially
    train = df.iloc[:train_end]
    test = df.iloc[train_end:test_end]
    validation = df.iloc[test_end:]

    return train, test, validation

In [6]:
import matplotlib.pyplot as plt

def plot_zoomed_anomalies(df, start_date, end_date, pollutant):
    anomaly_col = "Anomaly"  # Fixed syntax error

    if anomaly_col not in df.columns:
        raise KeyError(f"No anomaly column found. Available columns: {df.columns}")
    
    # Filter DataFrame for the specified date range
    df_filtered = df.loc[start_date:end_date]
    
    plt.figure(figsize=(12, 6))
    plt.plot(df_filtered.index, df_filtered[pollutant], label=pollutant, color='blue')
    
    # Highlight anomalies using the found anomaly column
    anomalies = df_filtered[df_filtered[anomaly_col] == 1]
    plt.scatter(anomalies.index, anomalies[pollutant], color='red', label="Anomaly", marker='o')
    
    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.title(f"Zoomed Anomalies for {pollutant}")
    plt.legend()
    plt.grid()
    plt.show()


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def process_and_plot_anomalies(df, pollutant):
    """
    Anomaly detection pipeline for PM2.5 or PM10 using Isolation Forest.
    
    Parameters:
      - df: DataFrame containing the air quality data.
      - pollutant: Either "pm2_5cnc" or "pm10cnc".
      
    Returns:
      - Updated DataFrame with anomaly labels.
    """

    # Validate pollutant
    if pollutant not in ["pm2_5cnc", "pm10cnc"]:
        raise ValueError("Invalid pollutant. Choose 'pm2_5cnc' or 'pm10cnc'.")

    # Define anomaly column name (e.g., "Anomaly_pm10cnc")
    anomaly_col = f"Anomaly_{pollutant.replace('.', '')}"

    # 🛠️ Data Cleaning & Preparation
    df_clean = clean_and_prepare(df)
    df_clean = df_clean.infer_objects(copy=False)  # Fix FutureWarning

    # Ensure datetime index (reset MultiIndex if needed)
    if isinstance(df_clean.index, pd.MultiIndex):
        df_for_split = df_clean.reset_index(level=1, drop=True)
    else:
        df_for_split = df_clean.copy()

    # 🏆 Split data (Train 70%, Test 20%, Validation 10%)
    train, test, validation = split_data_random(df_for_split)

    # Select the pollutant column
    train_data = train[[pollutant]]
    test_data = test[[pollutant]]
    val_data = validation[[pollutant]]

    # 🎯 Run Isolation Forest Model
    model, train_scores, train_labels = run_isoforest(train_data)
    _, test_scores, test_labels = run_isoforest(test_data)
    _, val_scores, val_labels = run_isoforest(val_data)

    # Convert anomaly labels: Isolation Forest returns -1 for anomalies.
    train["Anomaly"] = (train_labels == -1).astype(int)
    test["Anomaly"] = (test_labels == -1).astype(int)
    validation["Anomaly"] = (val_labels == -1).astype(int)

    # Reset index so dt_time becomes a column (if it isn't already)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)
    validation.reset_index(inplace=True)
    df_clean.reset_index(inplace=True)

    # Combine anomaly data from train, test, and validation into a single DataFrame.
    anomalies_combined = pd.concat([
        train[["dt_time", "Anomaly"]],
        test[["dt_time", "Anomaly"]],
        validation[["dt_time", "Anomaly"]]
    ])

    # If a timestamp appears in more than one set, mark it as an anomaly if any of them is 1.
    anomalies_combined = anomalies_combined.groupby("dt_time").max().reset_index()

    # Merge the combined anomaly information back into df_clean.
    df_merged = pd.merge(df_clean, anomalies_combined, on="dt_time", how="left")
    df_merged["Anomaly"] = df_merged["Anomaly"].fillna(0).astype(int)

    # Rename the generic "Anomaly" column to the desired anomaly_col.
    df_merged.rename(columns={"Anomaly": anomaly_col}, inplace=True)

    # Optionally, set the dt_time column back as index.
    df_merged.set_index("dt_time", inplace=True)

    # # 📈 Plot anomalies
    # plt.figure(figsize=(12, 5))
    # plt.scatter(train["dt_time"], train[pollutant], c=train["Anomaly"], cmap="coolwarm", label="Train", s=15)
    # plt.title(f"{pollutant.upper()} Concentration Anomalies (Isolation Forest)")
    # plt.xlabel("Time")
    # plt.ylabel(f"{pollutant} Concentration")
    # plt.legend()
    # plt.show()

    print(f"Train anomalies ({pollutant}): {train['Anomaly'].sum()}")
    print(f"Test anomalies ({pollutant}): {test['Anomaly'].sum()}")
    print(f"Validation anomalies ({pollutant}): {validation['Anomaly'].sum()}")

    return df_merged


In [8]:
# df10 = process_and_plot_anomalies(df,"pm2_5cnc")

In [9]:
# import os
# import pandas as pd
# import json

# # Ensure required folders exist
# CSV_FOLDER = "csvs/"
# JSON_FOLDER = "jsons/"
# os.makedirs(JSON_FOLDER, exist_ok=True)

# # Dictionary to store all site data
# all_sites_data = {}

# # Loop through all CSV files in "csvs/" folder
# for filename in os.listdir(CSV_FOLDER):
#     if filename.endswith(".csv"):
#         file_path = os.path.join(CSV_FOLDER, filename)

#         # Load CSV
#         df = pd.read_csv(file_path)

#         # Ensure 'site_id' column exists
#         if "deviceid" not in df.columns:
#             print(f"⚠️ Skipping {filename}: No 'site_id' column found!")
#             continue

#         # Get unique site IDs in this file
#         site_ids = df["deviceid"].unique()

#         for site_id in site_ids:
#             site_df = df[df["deviceid"] == site_id]  # Filter data per site
            
#             # Run anomaly detection for both pollutants
#             df10_iforest = process_and_plot_anomalies(site_df, pollutant="pm10cnc")
#             df25_iforest = process_and_plot_anomalies(site_df, pollutant="pm2_5cnc")

#             # Extract required columns
#             df10_iforest = df10_iforest[['dt_time', 'pm10cnc', 'Anomaly_pm10cnc']]
#             df25_iforest = df25_iforest[['dt_time', 'pm2_5cnc', 'Anomaly_pm25cnc']]

#             # Convert dt_time to string for JSON
#             df10_iforest["dt_time"] = df10_iforest["dt_time"].astype(str)
#             df25_iforest["dt_time"] = df25_iforest["dt_time"].astype(str)

#             # Merge both DataFrames on "dt_time"
#             merged_df = pd.merge(df10_iforest, df25_iforest, on="dt_time", how="inner")

#             # Store data in dictionary format
#             site_data = {str(site_id): merged_df.to_dict(orient="records")}
#             all_sites_data[str(site_id)] = merged_df.to_dict(orient="records")

#             # Save as separate JSON file for this site
#             json_filename = os.path.join(JSON_FOLDER, f"{site_id}.json")
#             with open(json_filename, "w") as f:
#                 json.dump(site_data, f, indent=4)

#             print(f"✅ Saved: {json_filename}")

# # Save master JSON file mapping all site IDs to their data
# master_json_path = os.path.join(JSON_FOLDER, "all_sites.json")
# with open(master_json_path, "w") as f:
#     json.dump(all_sites_data, f, indent=4)

# print(f"🚀 Master JSON saved: {master_json_path}")


In [10]:
import os
import pandas as pd
import json

# Folder paths
CSV_FOLDER = "csvs/"
JSON_FOLDER = "jsons/"

# Ensure JSON folder exists
os.makedirs(JSON_FOLDER, exist_ok=True)

# Function to process all CSV files dynamically
def process_all_sites():
    """
    Reads all CSV files from `csvs/` folder, processes data for each site,
    detects anomalies, and stores output in `jsons/` folder.
    """
    site_files = [f for f in os.listdir(CSV_FOLDER) if f.endswith(".csv")]

    for file in site_files:
        site_id = file.split("_")[1]  # Extract site_id from filename
        file_path = os.path.join(CSV_FOLDER, file)

        print(f"Processing {file} (Site ID: {site_id})...")

        # Read CSV with proper date parsing
        df = pd.read_csv(file_path, parse_dates=["dt_time"], index_col="dt_time")

        # Ensure essential columns exist
        if not all(col in df.columns for col in ["pm2_5cnc", "pm10cnc"]):
            print(f"⚠️ Skipping {file}: Required columns missing.")
            continue

        # Run anomaly detection models
        df10iforest = process_and_plot_anomalies(df, pollutant="pm10cnc")
        df25iforest = process_and_plot_anomalies(df, pollutant="pm2_5cnc")

        # Merge PM10 and PM2.5 anomaly data
        merged_df = pd.merge(
            df10iforest[['pm10cnc', 'Anomaly_pm10cnc']], 
            df25iforest[['pm2_5cnc', 'Anomaly_pm2_5cnc']], 
            left_index=True, 
            right_index=True, 
            how="inner"
        )

        # Reset index for JSON output
        merged_df.reset_index(inplace=True)
        merged_df["dt_time"] = merged_df["dt_time"].astype(str)  # Convert datetime to string

        # Convert processed DataFrame to JSON structure
        site_json_data = {site_id: merged_df.to_dict(orient="records")}

        # Save JSON file
        json_file_path = os.path.join(JSON_FOLDER, f"{site_id}.json")
        with open(json_file_path, "w") as json_file:
            json.dump(site_json_data, json_file, indent=4)

        print(f"✅ JSON saved: {json_file_path}")

# Run the processing function
if __name__ == "__main__":
    process_all_sites()


Processing site_104_data.csv (Site ID: 104)...


C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.interpolate(method="time", inplace=True)
C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:59: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:60: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


Train anomalies (pm10cnc): 2074
Test anomalies (pm10cnc): 590
Validation anomalies (pm10cnc): 294


C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.interpolate(method="time", inplace=True)
C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:59: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:60: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


Train anomalies (pm2_5cnc): 2072
Test anomalies (pm2_5cnc): 592
Validation anomalies (pm2_5cnc): 296
✅ JSON saved: jsons/104.json
Processing site_106_data.csv (Site ID: 106)...


...

Train anomalies (pm2_5cnc): 2073
Test anomalies (pm2_5cnc): 592
Validation anomalies (pm2_5cnc): 297
✅ JSON saved: jsons/5963.json
Processing site_5964_data.csv (Site ID: 5964)...


C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.interpolate(method="time", inplace=True)
C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:59: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:60: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


Train anomalies (pm10cnc): 2064
Test anomalies (pm10cnc): 590
Validation anomalies (pm10cnc): 297


C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.interpolate(method="time", inplace=True)
C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:59: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
C:\Users\21til\AppData\Local\Temp\ipykernel_21288\2886358829.py:60: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


Train anomalies (pm2_5cnc): 2074
Test anomalies (pm2_5cnc): 593
Validation anomalies (pm2_5cnc): 297
✅ JSON saved: jsons/5964.json
